In [1]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

train_original=pd.read_csv('train.csv')
train_original.set_index('PassengerId',inplace=True)

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score, RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import powerlaw
from scipy.stats import uniform


def create_model(in_layer,neurons=128,activation='relu',dropout=0.25,optimizer='adam',shape='one'):
    model=None
    model = Sequential()
    #first, input, layer, always present
    model.add(Dense(neurons, input_dim=in_layer,activation=activation))
    model.add(Dropout(dropout))
    if(shape=='none'):
        None
    if(shape=='one'):
        model.add(Dense(neurons,activation=activation))
        model.add(Dropout(dropout))
    if(shape=='two'):
        model.add(Dense(neurons,activation=activation))
        model.add(Dropout(dropout))
        model.add(Dense(neurons,activation=activation))
        model.add(Dropout(dropout))
    #final layer, always present
    model.add(Dense(8, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [2]:
data_1 = pd.read_csv('data_raw_1.csv',index_col='PassengerId')
data_2 = pd.read_csv('data_raw_2.csv',index_col='PassengerId')
data_3 = pd.read_csv('data_raw_3.csv',index_col='PassengerId')

decks=['A_deck', 'B_deck', 'C_deck',
       'D_deck', 'E_deck', 'F_deck', 'G_deck', 'T_deck']
decks_survived=['A_deck', 'B_deck', 'C_deck',
       'D_deck', 'E_deck', 'F_deck', 'G_deck', 'T_deck','Survived','Missing_Embark','Missing_Deck','deck']



deck_keys=pd.DataFrame({'A_deck':[1,0,0,0,0,0,0,0],
                        'B_deck':[0,1,0,0,0,0,0,0],
                        'C_deck':[0,0,1,0,0,0,0,0],
                        'D_deck':[0,0,0,1,0,0,0,0],
                        'E_deck':[0,0,0,0,1,0,0,0],
                        'F_deck':[0,0,0,0,0,1,0,0],
                        'G_deck':[0,0,0,0,0,0,1,0],
                        'T_deck':[0,0,0,0,0,0,0,1],
                        'Missing_Deck':[0,0,0,0,0,0,0,0]})
def get_deck(row):
    for c in deck_keys.columns:
        if row[c]==1:
            return c

data_1['deck']=data_1.apply(get_deck,axis=1)
data_1_nodeck=data_1.loc[(data_1['deck'] == 'Missing_Deck')]
data_1_deck=data_1.loc[(data_1['deck'] != 'Missing_Deck')]

data_2['deck']=data_2.apply(get_deck,axis=1)
data_2_nodeck=data_2.loc[(data_2['deck'] == 'Missing_Deck')]
data_2_deck=data_2.loc[(data_2['deck'] != 'Missing_Deck')]

data_3['deck']=data_3.apply(get_deck,axis=1)
data_3_nodeck=data_3.loc[(data_3['deck'] == 'Missing_Deck')]
data_3_deck=data_3.loc[(data_3['deck'] != 'Missing_Deck')]

print(data_1.shape,"\n",data_2.shape,"\n",data_3.shape)

(1309, 21) 
 (1309, 27) 
 (1309, 31)


# Data 1

In [14]:
X=data_1_deck.drop(decks_survived,axis=1)
Y=data_1_deck[decks].as_matrix()#.reshape((-1,1))

classifier = None
classifier = KerasClassifier(build_fn=create_model,verbose=0)

param_distributions = {'epochs':[1000],
                       'batch_size':[1024],
                       'in_layer':[X.shape[1]],
                       'neurons' : [64,128,256],#[32,64,128],
                       'optimizer': ['adam','rmsprop','adadelta'],#['adam','adagrad','rmsprop','adadelta'],
                       'dropout' : np.linspace(0.1,0.9,33),#uniform(0.1,1)
                       'activation' : ['relu'],
                       'shape':['two']
                      }

rscv = RandomizedSearchCV(estimator=classifier,
                          param_distributions=param_distributions,
                          cv=5,
                          verbose=1,
                          n_iter=10)

rscv.fit(X,Y)
results_df=pd.DataFrame(rscv.cv_results_)

results_df.sort_values('rank_test_score',axis=0).drop(['param_epochs','std_score_time','std_fit_time','params','mean_score_time','split0_test_score',
       'split0_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score','param_batch_size','param_in_layer'],axis=1).head(10)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 38.5min finished


,mean_fit_time,mean_test_score,mean_train_score,param_activation,param_dropout,param_neurons,param_optimizer,param_shape,rank_test_score,std_test_score,std_train_score
5,37.377643,0.413559,0.708475,relu,0.65,256,adam,two,1,0.057327,0.021338
1,34.117088,0.410169,0.577119,relu,0.625,128,adam,two,2,0.042067,0.016822
8,37.380444,0.393220,0.873729,relu,0.2,128,adam,two,3,0.056112,0.015718
2,34.523459,0.389831,0.626271,relu,0.35,64,adam,two,4,0.023970,0.024179
6,37.128407,0.389831,0.799153,relu,0.475,256,adadelta,two,4,0.028361,0.007859
7,35.912960,0.386441,0.427966,relu,0.725,64,rmsprop,two,6,0.047215,0.014678
4,34.606663,0.383051,0.681356,relu,0.5,128,rmsprop,two,7,0.031436,0.029037
9,38.722353,0.379661,0.795763,relu,0.325,128,rmsprop,two,8,0.025367,0.025900
0,34.180027,0.369492,0.619492,relu,0.55,128,adadelta,two,9,0.047215,0.015488
3,35.037899,0.338983,0.363559,relu,0.85,128,adam,two,10,0.061579,0.038594


# Data 2

In [16]:
X=data_2_deck.drop(decks_survived,axis=1)
Y=data_2_deck[decks].as_matrix()#.reshape((-1,1))

classifier = None
classifier = KerasClassifier(build_fn=create_model,verbose=0)

param_distributions = {'epochs':[1000],
                       'batch_size':[1024],
                       'in_layer':[X.shape[1]],
                       'neurons' : [64,128,256],#[32,64,128],
                       'optimizer': ['adam','rmsprop','adadelta'],#['adam','adagrad','rmsprop','adadelta'],
                       'dropout' : np.linspace(0.1,0.9,33),
                       'activation' : ['relu'],
                       'shape':['two']
                      }

rscv = RandomizedSearchCV(estimator=classifier,
                          param_distributions=param_distributions,
                          cv=5,
                          verbose=1,
                          n_iter=10)

rscv.fit(X,Y)
results_df=pd.DataFrame(rscv.cv_results_)

results_df.sort_values('rank_test_score',axis=0).drop(['param_epochs','std_score_time','std_fit_time','params','mean_score_time','split0_test_score',
       'split0_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score','param_batch_size','param_in_layer'],axis=1).head(10)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 59.3min finished


,mean_fit_time,mean_test_score,mean_train_score,param_activation,param_dropout,param_neurons,param_optimizer,param_shape,rank_test_score,std_test_score,std_train_score
3,50.751335,0.430508,0.964407,relu,0.25,256,rmsprop,two,1,0.057327,0.008725
9,58.744006,0.427119,0.945763,relu,0.425,256,rmsprop,two,2,0.042067,0.012964
5,56.273635,0.420339,0.711864,relu,0.625,128,rmsprop,two,3,0.043411,0.027591
2,52.760453,0.416949,0.912712,relu,0.325,128,rmsprop,two,4,0.020339,0.012455
1,51.883737,0.406780,0.859322,relu,0.425,128,adam,two,5,0.037134,0.015017
4,54.127163,0.400000,0.477119,relu,0.65,64,adam,two,6,0.069965,0.022834
0,52.576952,0.396610,0.456780,relu,0.675,64,adadelta,two,7,0.059298,0.025900
7,52.736733,0.386441,0.404237,relu,0.8,64,rmsprop,two,8,0.051854,0.027539
6,55.556544,0.379661,0.974576,relu,0.2,256,rmsprop,two,9,0.043675,0.002680
8,58.782127,0.308475,0.327966,relu,0.875,128,adadelta,two,10,0.056112,0.023759


# Data 3

In [17]:
X=data_3_deck.drop(decks_survived,axis=1)
Y=data_3_deck[decks].as_matrix()#.reshape((-1,1))

classifier = None
classifier = KerasClassifier(build_fn=create_model,verbose=0)


param_distributions = {'epochs':[1000],
                       'batch_size':[1024],
                       'in_layer':[X.shape[1]],
                       'neurons' : [64,128,256],#[32,64,128],
                       'optimizer': ['adam','rmsprop','adadelta'],#['adam','adagrad','rmsprop','adadelta'],
                       'dropout' : np.linspace(0.1,0.9,33),
                       'activation' : ['relu'],
                       'shape':['two']
                      }

rscv = RandomizedSearchCV(estimator=classifier,
                          param_distributions=param_distributions,
                          cv=5,
                          verbose=1,
                          n_iter=10)

rscv.fit(X,Y)
results_df=pd.DataFrame(rscv.cv_results_)

results_df.sort_values('rank_test_score',axis=0).drop(['param_epochs','std_score_time','std_fit_time','params','mean_score_time','split0_test_score',
       'split0_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score','param_batch_size','param_in_layer'],axis=1).head(10)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 64.7min finished


,mean_fit_time,mean_test_score,mean_train_score,param_activation,param_dropout,param_neurons,param_optimizer,param_shape,rank_test_score,std_test_score,std_train_score
2,64.457406,0.406780,0.949153,relu,0.35,256,adadelta,two,1,0.041517,0.011985
4,63.287188,0.396610,0.979661,relu,0.275,256,adam,two,2,0.052076,0.008642
6,49.931400,0.396610,0.905085,relu,0.375,128,adadelta,two,2,0.046231,0.010240
9,49.856280,0.383051,0.966102,relu,0.175,256,adadelta,two,4,0.008303,0.011985
0,56.504814,0.379661,0.544915,relu,0.6,64,adadelta,two,5,0.067457,0.028437
7,57.373727,0.372881,0.554237,relu,0.75,128,rmsprop,two,6,0.060639,0.025621
5,62.352793,0.352542,0.517797,relu,0.85,256,rmsprop,two,7,0.057126,0.018450
8,64.312437,0.335593,0.551695,relu,0.825,256,rmsprop,two,8,0.055078,0.009437
3,62.505146,0.332203,0.437288,relu,0.85,256,adadelta,two,9,0.036510,0.026038
1,57.700490,0.325424,0.338136,relu,0.875,128,adadelta,two,10,0.060068,0.022645


In [18]:
results_df.sort_values('rank_test_score',axis=0).drop(['param_epochs','std_score_time','std_fit_time','params','mean_score_time','split0_test_score',
       'split0_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score','param_batch_size','param_in_layer'],axis=1).tail(10)

,mean_fit_time,mean_test_score,mean_train_score,param_activation,param_dropout,param_neurons,param_optimizer,param_shape,rank_test_score,std_test_score,std_train_score
2,64.457406,0.406780,0.949153,relu,0.35,256,adadelta,two,1,0.041517,0.011985
4,63.287188,0.396610,0.979661,relu,0.275,256,adam,two,2,0.052076,0.008642
6,49.931400,0.396610,0.905085,relu,0.375,128,adadelta,two,2,0.046231,0.010240
9,49.856280,0.383051,0.966102,relu,0.175,256,adadelta,two,4,0.008303,0.011985
0,56.504814,0.379661,0.544915,relu,0.6,64,adadelta,two,5,0.067457,0.028437
7,57.373727,0.372881,0.554237,relu,0.75,128,rmsprop,two,6,0.060639,0.025621
5,62.352793,0.352542,0.517797,relu,0.85,256,rmsprop,two,7,0.057126,0.018450
8,64.312437,0.335593,0.551695,relu,0.825,256,rmsprop,two,8,0.055078,0.009437
3,62.505146,0.332203,0.437288,relu,0.85,256,adadelta,two,9,0.036510,0.026038
1,57.700490,0.325424,0.338136,relu,0.875,128,adadelta,two,10,0.060068,0.022645
